In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report


# --------------------------------------------------------------
# 1️⃣ LOAD DATA
# --------------------------------------------------------------
df = pd.read_csv("heart_attack_prediction_dataset.csv")


# --------------------------------------------------------------
# 2️⃣ REMOVE DUPLICATES
# --------------------------------------------------------------
df = df.drop_duplicates()


# --------------------------------------------------------------
# 3️⃣ SPLIT BLOOD PRESSURE INTO TWO NUMERIC COLUMNS
# --------------------------------------------------------------
df[["Systolic", "Diastolic"]] = df["Blood Pressure"].str.split("/", expand=True).astype(float)
df = df.drop("Blood Pressure", axis=1)


df["Sex"] = df["Sex"].map({"Male": 1, "Female": 0})
# --------------------------------------------------------------
# 4️⃣ FILL NULL AGE WITH MEAN AGE
# --------------------------------------------------------------
df["Age"] = df["Age"].fillna(df["Age"].mean())


# --------------------------------------------------------------
# 5️⃣ DROP UNWANTED COLUMNS
# --------------------------------------------------------------
cols_to_drop = [
    "Patient ID", "Income", "Country", "Alcohol Consumption", "Continent",
    "Hemisphere", "Physical Activity Days Per Week", "BMI",
    "Sedentary Hours Per Day"
]
df = df.drop(columns=cols_to_drop)


# --------------------------------------------------------------
# 6️⃣ ONE-HOT ENCODING FOR CATEGORICAL FEATURES
# --------------------------------------------------------------
df = pd.get_dummies(df, drop_first=True)


# --------------------------------------------------------------
# 7️⃣ DEFINE FEATURES & TARGET
# --------------------------------------------------------------
X = df.drop("Heart Attack Risk", axis=1)
y = df["Heart Attack Risk"]


# --------------------------------------------------------------
# 8️⃣ TRAIN / VALIDATION / TEST SPLIT (70/10/20)
# --------------------------------------------------------------
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, random_state=42
)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))

#-------------------------------------------------------------
# Convert Diet to dummy variables
df = pd.get_dummies(df, columns=['Diet'], drop_first=True)

# Convert True/False → 1/0
df = df.replace({True: 1, False: 0})

# Check
print(df[['Diet_Healthy', 'Diet_Unhealthy']].head())
# --------------------------------------------------------------
# 9️⃣ SCALE NUMERIC FEATURES
# --------------------------------------------------------------
X_train = X_train.values
X_val = X_val.values
X_test = X_test.values

# --------------------------------------------------------------
# 🔟 TRAIN RANDOM FOREST CLASSIFIER
# --------------------------------------------------------------
model = RandomForestClassifier(
    n_estimators=400,
    max_depth=12,
    class_weight="balanced",
    random_state=42
)

model.fit(X_train, y_train)


# --------------------------------------------------------------
# 1️⃣1️⃣ EVALUATE ON TEST SET
# --------------------------------------------------------------
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\n---- MODEL PERFORMANCE ----")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Train size: 6134
Validation size: 876
Test size: 1753


KeyError: "None of [Index(['Diet'], dtype='object')] are in the [columns]"

In [2]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", y_train_bal.value_counts())


Before SMOTE: Heart Attack Risk
0    3933
1    2201
Name: count, dtype: int64
After SMOTE: Heart Attack Risk
0    3933
1    3933
Name: count, dtype: int64


In [3]:
df["BP_Ratio"] = df["Systolic"] / df["Diastolic"]
df["Cholesterol_Ratio"] = df["Cholesterol"] / df["Age"]
df["Age_Risk"] = (df["Age"] > 50).astype(int)
df["High_BP"] = (df["Systolic"] > 140).astype(int)
df["High_Chol"] = (df["Cholesterol"] > 200).astype(int)


In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "max_depth": [4, 6, 8],
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [200, 400, 600],
    "subsample": [0.8, 1],
    "colsample_bytree": [0.8, 1]
}

xgb = XGBClassifier(eval_metric="logloss")

grid = GridSearchCV(
    xgb, params, scoring="accuracy", cv=5, n_jobs=-1
)

grid.fit(X_train_bal, y_train_bal)

print("Best Accuracy:", grid.best_score_)
print("Best Params:", grid.best_params_)


Best Accuracy: 0.717673235855054
Best Params: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.8}


In [5]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    grid.best_estimator_, X_train_bal, y_train_bal,
    cv=5, scoring='accuracy'
)

print("CV Accuracy:", scores.mean())


CV Accuracy: 0.717673235855054


In [12]:
df

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Exercise Hours Per Week,Previous Heart Problems,Medication Use,Stress Level,Triglycerides,Sleep Hours Per Day,Heart Attack Risk,Systolic,Diastolic,Diet_Healthy,Diet_Unhealthy
0,67,1,208,72,0,0,1,0,4.168189,0,0,9,286,6,0,158.0,88.0,False,False
1,21,1,389,98,1,1,1,1,1.813242,1,0,1,235,7,0,165.0,93.0,False,True
2,21,0,324,72,1,0,0,0,2.078353,1,1,9,587,4,0,174.0,99.0,True,False
3,84,1,383,73,1,1,1,0,9.828130,1,0,9,378,4,0,163.0,100.0,False,False
4,66,1,318,93,1,1,1,1,5.804299,1,0,6,231,5,0,91.0,88.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,60,1,121,61,1,1,1,0,7.917342,1,1,8,67,7,0,94.0,76.0,True,False
8759,28,0,120,73,1,0,0,1,16.558426,0,0,8,617,9,0,157.0,102.0,True,False
8760,47,1,250,105,0,1,1,1,3.148438,1,0,5,527,4,1,161.0,75.0,False,False
8761,36,1,178,60,1,0,1,0,3.789950,1,1,5,114,8,0,119.0,67.0,False,True
